In [1]:
import mimetypes
import pathlib
import shutil
import tempfile

from PIL import Image, ImageOps
# from  pillow_heif import register_heif_opener.  # for mac images
# register_heif_opener()



In [2]:
# pathlib.Path(__file__).resolve() # For python file
NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent
DATA_DIR = REPO_DIR / 'data'
INPUT_DIR = DATA_DIR / 'inputs'
READY_DIR = DATA_DIR / 'ready'
OUTPUT_DIR = DATA_DIR / 'outputs'

In [3]:
def perform_clear_and_optimize_image(image_path, output_path, max_size=(1920,1920)):

    """
    Removes all metadata from the image ( eg. EXIF data)
    Optimizes the image file size while preserving quality and transparency of the image
    :param image_path:
    :param output_path:
    :param max_size: Fixed size
    :return:
    """

    # convert to path object
    input_path = pathlib.Path(image_path)
    output_path = pathlib.Path(output_path)

    # Open and create a clean copy
    original = Image.open(image_path)

    # Determine if image has transparency
    has_transparency = original.mode in ('RGBA', 'LA') or  (original.mode == 'P' and 'transparency' in original.info)

    # Auto-rotate based on EXIF
    original = ImageOps.exif_transpose(original)

    # Resize if larger than the max_size while maintaining all aspects
    if original.size[0]>max_size[0] or original.size[1]>max_size[1]:
        original.thumbnail(max_size,Image.Resampling.LANCZOS)

    # Convert mode based on transparency
    if has_transparency:
        if original.mode != 'RGBA':
            original = original.convert('RGBA')
        best_format = 'PNG'
    else:
        if original.mode in ("RGBA", "P", "LA"):
            original = original.convert('RGB')
        best_format = 'JPEG'

    # save with optimized settings
    save_kwargs = {}
    if best_format == 'JPEG':
        save_kwargs.update({
           'quality': 85,
            'optimize': True,
            'progressive': True,
        })
        output_path = output_path.with_suffix('.jpg')
    elif best_format == 'PNG':
        save_kwargs.update({
            'optimize': True,
            'compress_level': 6,
        })
        output_path = output_path.with_suffix('.png')
    print(f"Saving {output_path}")
    original.save(output_path, format=best_format, **save_kwargs)
    return output_path




    # image = Image.open(image_path)
    # image = ImageOps.exif_transpose(image)
    # image = ImageOps.autocontrast(image)
    # image = ImageOps.equalize(image)
    # image = ImageOps.posterize(image, 4)
    # image = ImageOps.solarize(image, threshold=128)
    # image = ImageOps.invert(image)
    # image = ImageOps.grayscale(image)
    # image = ImageOps.colorize(image, black="black", white="white")

In [13]:

def perform_is_image(path, require_can_open=True):

    try:
        guess_type, encoding = mimetypes.guess_type(path)
        guess_type = guess_type or ""
    except:
        guess_type = ""
    print(file_path.name, guess_type)
    guessed_image = "image" in guess_type
    if not guessed_image:
        return False
    if guessed_image and require_can_open:
        try:
            img_ = Image.open(path)
            print(img_)
        except:
            return False
    return True

In [14]:
image_file_paths = []

for file_path in INPUT_DIR.glob('*'):
    print(file_path)
    is_image = perform_is_image(file_path)
    if not is_image:
        continue
    start_op_path = READY_DIR / file_path.name
    final_op_path = perform_clear_and_optimize_image(file_path, start_op_path)
    image_file_paths.append(final_op_path)

image_file_paths

/Users/slackroo/Data_science/frontend_stack/superme-api/data/inputs/not-an-image.txt
not-an-image.txt text/plain
/Users/slackroo/Data_science/frontend_stack/superme-api/data/inputs/.DS_Store
.DS_Store 
/Users/slackroo/Data_science/frontend_stack/superme-api/data/inputs/8.jpg
8.jpg image/jpeg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4000x1868 at 0x108E426F0>
Saving /Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/8.jpg
/Users/slackroo/Data_science/frontend_stack/superme-api/data/inputs/1.jpeg
1.jpeg image/jpeg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1792x1195 at 0x108A54CB0>
Saving /Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/1.jpg
/Users/slackroo/Data_science/frontend_stack/superme-api/data/inputs/9.jpg
9.jpg image/jpeg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=959x1468 at 0x1089850D0>
Saving /Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/9.jpg
/Users/slackroo/Data_science/frontend_sta

[PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/8.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/1.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/9.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/11.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/10.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/4.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/5.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/7.png'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/6.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/2.jpg'),
 PosixPath('/Users/slackroo/Data_science/frontend_stack/superme-api/data/ready/3.jpg')]

In [15]:
READY_DIR.mkdir(exist_ok=True, parents=True)
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [16]:
zip_outpath = OUTPUT_DIR / 'images-optimized.zip'
zip_outpath.exists()

True

In [17]:
with tempfile.TemporaryDirectory() as temp_dir:
    for path in image_file_paths:
        shutil.copy(path, temp_dir)

    shutil.make_archive(zip_outpath.with_suffix(''), 'zip', temp_dir)